In [1]:
import pandas as pd
import re
import warnings
from fuzzywuzzy import fuzz
warnings.filterwarnings('ignore')
pd.set_option('max_columns',151)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
outfield = pd.read_csv('data/2019-20_out.csv')
fpl = pd.read_csv('data/cleaned_players.csv')

In [3]:
for c in outfield.columns:
    print(f"'{c}'",end=',')

'player','nationality','position','squad','age','birth_year','games','games_starts','minutes','goals','assists','pens_made','pens_att','cards_yellow','cards_red','goals_per90','assists_per90','goals_assists_per90','goals_pens_per90','goals_assists_pens_per90','xg','npxg','xa','xg_per90','xa_per90','xg_xa_per90','npxg_per90','npxg_xa_per90','minutes_90s','shots_total','shots_on_target','shots_free_kicks','shots_on_target_pct','shots_total_per90','shots_on_target_per90','goals_per_shot','goals_per_shot_on_target','npxg_per_shot','xg_net','npxg_net','passes_completed','passes','passes_pct','passes_total_distance','passes_progressive_distance','passes_completed_short','passes_short','passes_pct_short','passes_completed_medium','passes_medium','passes_pct_medium','passes_completed_long','passes_long','passes_pct_long','xa_net','assisted_shots','passes_into_final_third','passes_into_penalty_area','crosses_into_penalty_area','progressive_passes','passes_live','passes_dead','passes_free_kicks'

In [4]:
defender = outfield.loc[:,['player','position','squad','games','minutes','cards_yellow','cards_red','goals_per90',
                           'assists_per90','goals_assists_per90','xg_per90','xa_per90','xg_xa_per90',
                           'tackles','tackles_won','dribble_tackles_pct','pressure_regain_pct',
                           'blocks','interceptions','clearances']]
defender = defender[defender['position']=='DF']
relegated = ['Bournemoth','Norwich City','Watford']
defender = defender[~defender['squad'].isin(relegated)]
defender['tackle_perc'] = None
defender['block_int_clear'] = None
defender['tackle_perc'] = defender.apply(lambda row: (row['tackles_won']/row['tackles']) if row['tackles']!=0 else 0, axis=1)
defender['block_int_clear'] = defender.apply(lambda row:(row['blocks']+row['interceptions']+row['clearances']),axis=1)
defender = defender.drop(columns=['tackles','tackles_won','blocks','interceptions','clearances']).reset_index(drop=True)
defender.head()

,player,position,squad,games,minutes,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,xg_per90,xa_per90,xg_xa_per90,dribble_tackles_pct,pressure_regain_pct,tackle_perc,block_int_clear
0,Patrick van Aanholt,DF,Crystal Palace,29.0,2507.0,0.0,0.0,0.11,0.07,0.18,0.08,0.11,0.20,40.7,27.7,0.627451,170.0
1,Nathan Aké,DF,Bournemouth,29.0,2503.0,2.0,0.0,0.07,0.07,0.14,0.09,0.02,0.11,29.2,29.7,0.440000,268.0
2,Toby Alderweireld,DF,Tottenham,33.0,2957.0,7.0,0.0,0.06,0.06,0.12,0.03,0.03,0.07,38.9,32.2,0.545455,324.0
3,Trent Alexander-Arnold,DF,Liverpool,38.0,3175.0,5.0,0.0,0.11,0.37,0.48,0.08,0.27,0.36,40.0,32.0,0.431034,197.0
4,Marcos Alonso,DF,Chelsea,18.0,1430.0,4.0,0.0,0.25,0.13,0.38,0.13,0.13,0.26,17.9,40.9,0.703704,98.0


In [5]:
dup_def = defender[defender.duplicated(['player'],keep=False)]
dup_def = dup_def.groupby(['player'], as_index=False).agg('sum')
dup_def.insert(1,'position','DF')
dup_squad = ['Southampton + Arsenal','Newcastle Utd + Tottenham','Tottenham + Southampton','Wolves + Leicester City','Chelsea + Brighton']
dup_def.insert(2,'squad',dup_squad)
dup_def

,player,position,squad,games,minutes,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,xg_per90,xa_per90,xg_xa_per90,dribble_tackles_pct,pressure_regain_pct,tackle_perc,block_int_clear
0,Cédric Soares,DF,Southampton + Arsenal,21.0,1555.0,1.0,0.0,0.36,0.07,0.43,0.05,0.11,0.15,87.4,58.2,1.355072,129.0
1,Danny Rose,DF,Newcastle Utd + Tottenham,23.0,1756.0,7.0,0.0,0.00,0.00,0.00,0.02,0.09,0.10,86.2,64.5,1.389423,105.0
2,Kyle Walker-Peters,DF,Tottenham + Southampton,13.0,944.0,1.0,0.0,0.00,0.00,0.00,0.02,0.10,0.12,53.3,62.9,1.171429,82.0
3,Ryan Bennett,DF,Wolves + Leicester City,16.0,937.0,2.0,0.0,0.00,0.00,0.00,0.00,0.02,0.02,95.3,44.6,1.150000,94.0
4,Tariq Lamptey,DF,Chelsea + Brighton,9.0,620.0,2.0,0.0,0.00,0.15,0.15,0.01,0.43,0.44,100.0,118.7,0.333333,46.0


In [6]:
defender = defender.drop_duplicates(subset='player',keep=False)
defender = pd.concat([defender,dup_def])
defender.head()

,player,position,squad,games,minutes,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,xg_per90,xa_per90,xg_xa_per90,dribble_tackles_pct,pressure_regain_pct,tackle_perc,block_int_clear
0,Patrick van Aanholt,DF,Crystal Palace,29.0,2507.0,0.0,0.0,0.11,0.07,0.18,0.08,0.11,0.20,40.7,27.7,0.627451,170.0
1,Nathan Aké,DF,Bournemouth,29.0,2503.0,2.0,0.0,0.07,0.07,0.14,0.09,0.02,0.11,29.2,29.7,0.440000,268.0
2,Toby Alderweireld,DF,Tottenham,33.0,2957.0,7.0,0.0,0.06,0.06,0.12,0.03,0.03,0.07,38.9,32.2,0.545455,324.0
3,Trent Alexander-Arnold,DF,Liverpool,38.0,3175.0,5.0,0.0,0.11,0.37,0.48,0.08,0.27,0.36,40.0,32.0,0.431034,197.0
4,Marcos Alonso,DF,Chelsea,18.0,1430.0,4.0,0.0,0.25,0.13,0.38,0.13,0.13,0.26,17.9,40.9,0.703704,98.0


In [7]:
defender.shape

(160, 17)

In [8]:
fpl

,first_name,second_name,goals_scored,assists,total_points,minutes,goals_conceded,creativity,influence,threat,bonus,bps,ict_index,clean_sheets,red_cards,yellow_cards,selected_by_percent,now_cost,element_type
0,Mesut,Özil,1,3,53,1439,20,582.9,223.6,190.0,1,256,99.8,5,0,1,1.9,70,MID
1,Sokratis,Papastathopoulos,2,0,57,1696,25,36.8,436.2,110.0,5,305,58.5,4,0,6,0.5,50,DEF
2,David,Luiz Moreira Marinho,2,1,94,2809,42,106.7,701.6,211.0,10,494,102.1,8,2,5,2.8,55,DEF
3,Pierre-Emerick,Aubameyang,22,5,205,3136,44,479.6,1006.0,1369.0,37,807,285.2,10,1,3,43.3,120,MID
4,Cédric,Soares,1,1,61,1553,20,218.9,349.0,118.0,3,286,68.7,4,0,1,0.6,50,DEF
5,Alexandre,Lacazette,10,7,118,1867,25,337.7,536.4,861.0,16,387,173.8,5,0,8,2.8,85,FWD
6,Shkodran,Mustafi,0,2,43,1205,17,45.5,277.2,155.0,2,242,47.9,4,0,2,0.0,50,DEF
7,Bernd,Leno,0,0,114,2649,39,0.0,843.2,0.0,10,591,84.1,7,0,2,6.3,50,GK
8,Granit,Xhaka,1,2,73,2586,31,298.5,486.2,110.0,2,408,89.9,10,0,10,0.6,55,MID
9,Pablo,Marí,0,0,8,113,0,1.7,27.8,0.0,1,31,3.0,1,0,0,0.0,45,DEF


In [9]:
fpl_def = fpl[fpl['element_type']=='DEF']
fpl_def['cost'] = None
fpl_def['player'] = None
fpl_def['player'] = fpl_def['first_name'].str.cat(fpl_def['second_name'],sep=' ')

fpl_def = fpl_def.loc[:,['second_name','player','total_points','now_cost']].reset_index(drop=True)
fpl_def.head()

,second_name,player,total_points,now_cost
0,Papastathopoulos,Sokratis Papastathopoulos,57,50
1,Luiz Moreira Marinho,David Luiz Moreira Marinho,94,55
2,Soares,Cédric Soares,61,50
3,Mustafi,Shkodran Mustafi,43,50
4,Marí,Pablo Marí,8,45


In [10]:
cost = {}
points = {}

for def_p in defender.itertuples():
    for fpl_p in fpl_def.itertuples():
        if (fuzz.token_set_ratio(def_p[1],fpl_p[2])>80):
            print(def_p[1],fpl_p[3],fpl_p[4]/10)
            points[def_p[1]] = fpl_p[3]
            cost[def_p[1]] = fpl_p[4]/10
            break

Patrick van Aanholt 109 5.5
Nathan Aké 79 5.5
Toby Alderweireld 97 5.5
Trent Alexander-Arnold 210 7.5
Marcos Alonso 100 6.0
Serge Aurier 124 5.5
César Azpilicueta 130 6.0
Eric Bailly 9 5.0
Fabián Balbuena 40 4.5
George Baldock 142 5.5
Phil Bardsley 78 4.5
Chris Basham 123 5.0
Jan Bednarek 84 4.5
Héctor Bellerín 44 5.0
Bernardo 21 4.0
Ryan Bertrand 75 5.0
Willy Boly 97 5.5
Jarrad Branthwaite 11 4.0
Dan Burn 86 4.5
Gary Cahill 70 4.5
João Cancelo 45 5.5
Calum Chambers 42 4.5
Ben Chilwell 114 5.5
Andreas Christensen 34 5.0
Ciaran Clark 49 4.5
Conor Coady 118 5.0
Séamus Coleman 55 5.0
Aaron Cresswell 79 5.0
Diogo Dalot 4 5.0
Scott Dann 51 4.5
Ben Davies 41 5.0
Lucas Digne 120 6.0
Virgil van Dijk 178 6.5
Issa Diop 76 4.5
Matt Doherty 167 6.0
Shane Duffy 44 4.5
Paul Dummett 44 4.5
Lewis Dunk 128 5.0
John Egan 133 5.0
Ahmed Elmohamady 39 4.5
Björn Engels 42 4.5
Jonny Evans 128 5.5
Federico Fernández 102 5.0
Timothy Fosu-Mensah 7 4.5
Juan Foyth 3 4.5
Ryan Fredericks 74 4.5
Christian Fuchs 26 4

In [11]:
points

{'Patrick van Aanholt': 109,
 'Nathan Aké': 79,
 'Toby Alderweireld': 97,
 'Trent Alexander-Arnold': 210,
 'Marcos Alonso': 100,
 'Serge Aurier': 124,
 'César Azpilicueta': 130,
 'Eric Bailly': 9,
 'Fabián Balbuena': 40,
 'George Baldock': 142,
 'Phil Bardsley': 78,
 'Chris Basham': 123,
 'Jan Bednarek': 84,
 'Héctor Bellerín': 44,
 'Bernardo': 21,
 'Ryan Bertrand': 75,
 'Willy Boly': 97,
 'Jarrad Branthwaite': 11,
 'Dan Burn': 86,
 'Gary Cahill': 70,
 'João Cancelo': 45,
 'Calum Chambers': 42,
 'Ben Chilwell': 114,
 'Andreas Christensen': 34,
 'Ciaran Clark': 49,
 'Conor Coady': 118,
 'Séamus Coleman': 55,
 'Aaron Cresswell': 79,
 'Diogo Dalot': 4,
 'Scott Dann': 51,
 'Ben Davies': 41,
 'Lucas Digne': 120,
 'Virgil van Dijk': 178,
 'Issa Diop': 76,
 'Matt Doherty': 167,
 'Shane Duffy': 44,
 'Paul Dummett': 44,
 'Lewis Dunk': 128,
 'John Egan': 133,
 'Ahmed Elmohamady': 39,
 'Björn Engels': 42,
 'Jonny Evans': 128,
 'Federico Fernández': 102,
 'Timothy Fosu-Mensah': 7,
 'Juan Foyth': 3

In [12]:
cost

{'Patrick van Aanholt': 5.5,
 'Nathan Aké': 5.5,
 'Toby Alderweireld': 5.5,
 'Trent Alexander-Arnold': 7.5,
 'Marcos Alonso': 6.0,
 'Serge Aurier': 5.5,
 'César Azpilicueta': 6.0,
 'Eric Bailly': 5.0,
 'Fabián Balbuena': 4.5,
 'George Baldock': 5.5,
 'Phil Bardsley': 4.5,
 'Chris Basham': 5.0,
 'Jan Bednarek': 4.5,
 'Héctor Bellerín': 5.0,
 'Bernardo': 4.0,
 'Ryan Bertrand': 5.0,
 'Willy Boly': 5.5,
 'Jarrad Branthwaite': 4.0,
 'Dan Burn': 4.5,
 'Gary Cahill': 4.5,
 'João Cancelo': 5.5,
 'Calum Chambers': 4.5,
 'Ben Chilwell': 5.5,
 'Andreas Christensen': 5.0,
 'Ciaran Clark': 4.5,
 'Conor Coady': 5.0,
 'Séamus Coleman': 5.0,
 'Aaron Cresswell': 5.0,
 'Diogo Dalot': 5.0,
 'Scott Dann': 4.5,
 'Ben Davies': 5.0,
 'Lucas Digne': 6.0,
 'Virgil van Dijk': 6.5,
 'Issa Diop': 4.5,
 'Matt Doherty': 6.0,
 'Shane Duffy': 4.5,
 'Paul Dummett': 4.5,
 'Lewis Dunk': 5.0,
 'John Egan': 5.0,
 'Ahmed Elmohamady': 4.5,
 'Björn Engels': 4.5,
 'Jonny Evans': 5.5,
 'Federico Fernández': 5.0,
 'Timothy Fosu

In [13]:
defender['cost'] = defender['player'].map(cost)
defender['points'] = defender['player'].map(points)
defender = defender.reset_index(drop=True)
defender

,player,position,squad,games,minutes,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,xg_per90,xa_per90,xg_xa_per90,dribble_tackles_pct,pressure_regain_pct,tackle_perc,block_int_clear,cost,points
0,Patrick van Aanholt,DF,Crystal Palace,29.0,2507.0,0.0,0.0,0.11,0.07,0.18,0.08,0.11,0.20,40.7,27.7,0.627451,170.0,5.5,109.0
1,Nathan Aké,DF,Bournemouth,29.0,2503.0,2.0,0.0,0.07,0.07,0.14,0.09,0.02,0.11,29.2,29.7,0.440000,268.0,5.5,79.0
2,Toby Alderweireld,DF,Tottenham,33.0,2957.0,7.0,0.0,0.06,0.06,0.12,0.03,0.03,0.07,38.9,32.2,0.545455,324.0,5.5,97.0
3,Trent Alexander-Arnold,DF,Liverpool,38.0,3175.0,5.0,0.0,0.11,0.37,0.48,0.08,0.27,0.36,40.0,32.0,0.431034,197.0,7.5,210.0
4,Marcos Alonso,DF,Chelsea,18.0,1430.0,4.0,0.0,0.25,0.13,0.38,0.13,0.13,0.26,17.9,40.9,0.703704,98.0,6.0,100.0
5,Angeliño,DF,Manchester City,6.0,423.0,1.0,0.0,0.00,0.00,0.00,0.05,0.26,0.31,83.3,38.3,0.625000,31.0,NaN,NaN
6,Serge Aurier,DF,Tottenham,33.0,2700.0,4.0,1.0,0.03,0.17,0.20,0.02,0.10,0.12,52.5,29.4,0.561224,206.0,5.5,124.0
7,César Azpilicueta,DF,Chelsea,36.0,3229.0,1.0,0.0,0.06,0.17,0.22,0.07,0.16,0.23,50.0,32.7,0.598039,293.0,6.0,130.0
8,Eric Bailly,DF,Manchester Utd,4.0,150.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,23.5,0.000000,13.0,5.0,9.0
9,Leighton Baines,DF,Everton,8.0,404.0,0.0,0.0,0.00,0.00,0.00,0.01,0.18,0.19,28.6,27.5,0.600000,28.0,NaN,NaN


In [14]:
defender[defender.isna().any(axis=1)]

,player,position,squad,games,minutes,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,xg_per90,xa_per90,xg_xa_per90,dribble_tackles_pct,pressure_regain_pct,tackle_perc,block_int_clear,cost,points
5,Angeliño,DF,Manchester City,6.0,423.0,1.0,0.0,0.00,0.00,0.00,0.05,0.26,0.31,83.3,38.3,0.625000,31.0,NaN,NaN
9,Leighton Baines,DF,Everton,8.0,404.0,0.0,0.0,0.00,0.00,0.00,0.01,0.18,0.19,28.6,27.5,0.600000,28.0,NaN,NaN
23,Jonny Castro,DF,Wolves,35.0,2887.0,4.0,0.0,0.06,0.06,0.12,0.04,0.03,0.07,37.8,29.8,0.622449,176.0,NaN,NaN
30,Steve Cook,DF,Bournemouth,29.0,2428.0,6.0,1.0,0.04,0.00,0.04,0.03,0.01,0.04,39.1,33.5,0.590909,314.0,NaN,NaN
33,Charlie Daniels,DF,Bournemouth,2.0,126.0,0.0,0.0,0.00,0.00,0.00,0.00,0.05,0.05,50.0,19.2,0.000000,10.0,NaN,NaN
35,Kevin Danso,DF,Southampton,6.0,277.0,2.0,1.0,0.00,0.32,0.32,0.03,0.07,0.10,62.5,24.4,0.500000,27.0,NaN,NaN
45,Harvey Elliot,DF,Liverpool,2.0,8.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,NaN,NaN
50,Fernandinho,DF,Manchester City,30.0,2397.0,9.0,2.0,0.00,0.04,0.04,0.04,0.02,0.06,38.3,41.0,0.568182,171.0,NaN,NaN
53,Simon Francis,DF,Bournemouth,15.0,969.0,5.0,1.0,0.00,0.00,0.00,0.02,0.03,0.04,44.0,27.1,0.705882,140.0,NaN,NaN
62,Jordon Ibe,DF,Bournemouth,2.0,50.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,27.3,1.000000,0.0,NaN,NaN


In [15]:
defender['ppg'] = defender.apply(lambda row: (row['points']/row['games']),axis=1)
defender['ppc'] = defender.apply(lambda row: (row['points']/row['cost']),axis=1)
defender = defender.dropna().reset_index(drop=True)
defender

,player,position,squad,games,minutes,cards_yellow,cards_red,goals_per90,assists_per90,goals_assists_per90,xg_per90,xa_per90,xg_xa_per90,dribble_tackles_pct,pressure_regain_pct,tackle_perc,block_int_clear,cost,points,ppg,ppc
0,Patrick van Aanholt,DF,Crystal Palace,29.0,2507.0,0.0,0.0,0.11,0.07,0.18,0.08,0.11,0.20,40.7,27.7,0.627451,170.0,5.5,109.0,3.758621,19.818182
1,Nathan Aké,DF,Bournemouth,29.0,2503.0,2.0,0.0,0.07,0.07,0.14,0.09,0.02,0.11,29.2,29.7,0.440000,268.0,5.5,79.0,2.724138,14.363636
2,Toby Alderweireld,DF,Tottenham,33.0,2957.0,7.0,0.0,0.06,0.06,0.12,0.03,0.03,0.07,38.9,32.2,0.545455,324.0,5.5,97.0,2.939394,17.636364
3,Trent Alexander-Arnold,DF,Liverpool,38.0,3175.0,5.0,0.0,0.11,0.37,0.48,0.08,0.27,0.36,40.0,32.0,0.431034,197.0,7.5,210.0,5.526316,28.000000
4,Marcos Alonso,DF,Chelsea,18.0,1430.0,4.0,0.0,0.25,0.13,0.38,0.13,0.13,0.26,17.9,40.9,0.703704,98.0,6.0,100.0,5.555556,16.666667
5,Serge Aurier,DF,Tottenham,33.0,2700.0,4.0,1.0,0.03,0.17,0.20,0.02,0.10,0.12,52.5,29.4,0.561224,206.0,5.5,124.0,3.757576,22.545455
6,César Azpilicueta,DF,Chelsea,36.0,3229.0,1.0,0.0,0.06,0.17,0.22,0.07,0.16,0.23,50.0,32.7,0.598039,293.0,6.0,130.0,3.611111,21.666667
7,Eric Bailly,DF,Manchester Utd,4.0,150.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,23.5,0.000000,13.0,5.0,9.0,2.250000,1.800000
8,Fabián Balbuena,DF,West Ham,17.0,1177.0,2.0,0.0,0.08,0.00,0.08,0.14,0.00,0.14,23.1,34.5,0.611111,146.0,4.5,40.0,2.352941,8.888889
9,George Baldock,DF,Sheffield Utd,38.0,3420.0,7.0,0.0,0.05,0.08,0.13,0.02,0.07,0.09,52.1,28.0,0.608108,187.0,5.5,142.0,3.736842,25.818182


In [16]:
defender.to_csv('data/defender_dash.csv',index=False)